In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
from matplotlib import pyplot as plt
import sys
import matplotlib.animation as animation
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import scipy.spatial.distance as dist
import numpy as np
import h5py
from tqdm import tqdm
import pandas as pd
import imageio

#import tables
#import deepdish as dd

sys.path.append('/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/post_cmfe_analysis')
import python_utils_jjm as jjm_utils

In [3]:
results_file = '/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/memmap_0000memmap_0004_resized_source_extraction_13:05:33.239/frames_1_5000/LOGS_15-Oct_13_10_52/15-Oct_13_23_16.mat_out.mat'
results = sio.loadmat(results_file)

##A is spatial contours

In [5]:
#img_file = '/Users/johnmarshall/Documents/Analysis/MiniscopeMovies/GRIN011/H10_M19_S59msCam1.tif'
#img_from_file = plt.imread(img_file)    
#d1, d2 = np.shape(img_from_file)
d1, d2 = 480, 752

In [9]:
dense_A = results['A'].todense()

In [10]:
frames = np.shape(results['C'])[0]

In [11]:
cells = np.shape(results['C'])[0]

In [20]:
#reshape for all frames
frame_range = (0, 150)
cells_reshaped = np.empty((cells, frame_range[1], d1, d2))
for cell in tqdm(range(cells)):
    A_reshaped = np.reshape(dense_A[:, cell], (d1, d2))
    cells_reshaped[cell] = np.array([np.dot(A_reshaped, results['C'][cell, frame]) for frame in range(frame_range[0], frame_range[1])])
    


100%|██████████| 120/120 [02:10<00:00,  1.08s/it]


In [25]:
hf = h5py.File('/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/memmap_0000memmap_0004_resized_source_extraction_13:05:33.239/frames_1_5000/LOGS_15-Oct_13_10_52/demixed_movie.h5', 'w')

In [26]:
hf.create_dataset('demixed_movie_array', data=cells_reshaped )

<HDF5 dataset "demixed_movie_array": shape (120, 150, 480, 752), type "<f8">

In [28]:
hf.close()

In [35]:
cells_recombined = np.sum(cells_reshaped, axis=0)

In [42]:
imageio.mimwrite('/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/demixed.mp4', cells_recombined, fps = 20)

Lossy conversion from float64 to uint8. Range [0.0, 17.144301860839185]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 16.00869305994339]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 14.949259841160604]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 13.986830864705937]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 13.08688013577738]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 12.245320261442743]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 14.734950994241487]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uin

Lossy conversion from float64 to uint8. Range [0.0, 7.507083770256033]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.740090647622211]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.275876984423345]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 7.381595630101776]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 6.906138701809837]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 6.4904741954637695]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 9.016179586415118]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. 

Lossy conversion from float64 to uint8. Range [0.0, 35.27699298234868]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 33.57909500532493]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 34.6339344883619]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 32.064664202533095]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 29.68720416377915]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 27.487146270325795]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0.0, 25.876426321373618]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8.

In [5]:
## recombine demixed video with behavior data 
cnmfe_range = (0, 150)
dir_path = '/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/'
frame_clock_info = 'timestamp.dat'
frame_clock_df = pd.read_table(dir_path+frame_clock_info)

# load time stamps 
msCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 0].set_index('frameNum')[cnmfe_range[0]:cnmfe_range[1]]
behavCam_timestamps = frame_clock_df[frame_clock_df['camNum'] == 1].set_index('frameNum')
# reset initial clock value to 0 
msCam_timestamps['sysClock'][1] = 0
behavCam_timestamps['sysClock'][1] = 0

/Users/johnmarshall/miniconda3/envs/caiman/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """


In [7]:
# create df to find matched behavior cam frames
msCam_timestamps = jjm_utils.align_behavior_data(msCam_timestamps, behavCam_timestamps)
msCam_timestamps.reset_index(inplace=True)
msCam_timestamps.head()

100%|██████████| 150/150 [00:01<00:00, 89.27it/s]


,frameNum,camNum,sysClock,buffer,behavCam_frames,sys_clock_behavCam
0,1,0,0,1,1,0
1,2,0,7,1,1,0
2,3,0,58,1,3,47
3,4,0,108,1,5,111
4,5,0,158,1,6,143


In [9]:
# 
reader = imageio.get_reader('/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/behavCam1.avi')

In [18]:
bd1, bd2 = reader.get_meta_data()['size']

In [28]:
1 in msCam_timestamps['behavCam_frames'].values

True

In [35]:
behavior_adjusted = np.empty((cnmfe_range[1], bd2, bd1, 3))    
#for frame in range(cnmfe_range[0], cnmfe_range[1])
frame = 0
for i, im in enumerate(reader):
    if 1 in msCam_timestamps['behavCam_frames'].values:
        behavior_adjusted[0] = im
        

In [36]:
# combine those matched frames into a video

imageio.mimwrite('/Volumes/My_Passport/cnmfe_analysis_files/GRIN011/behavior_adjusted.mp4', behavior_adjusted, fps = 20)